In [1]:
#r "nuget: Microsoft.SemanticKernel.Agents.Abstractions, *-*"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, *-*"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, *-*"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureOpenAI, *-*"

Installed Packages Microsoft.SemanticKernel.Agents.Abstractions, 1.32.0-alpha Microsoft.SemanticKernel.Agents.Core, 1.32.0-alpha Microsoft.SemanticKernel.Agents.OpenAI, 1.32.0-alpha Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.32.0

In [2]:
using System.ComponentModel;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;

In [3]:
var deployment = "gpt-4o-mini";
var endpoint = "Your AOAI endpoint";
var key = "Your AOAI Key";

In [4]:
#pragma warning disable SKEXP0010

var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(deployment, endpoint, key)
    .Build();

In [6]:
#r "nuget: Azure.AI.Projects, 1.0.0-beta.2"

#r "nuget: Azure.Identity, 1.13.1"

Installed Packages Azure.AI.Projects, 1.0.0-beta.2 Azure.Identity, 1.13.1

In [7]:

using System;
using System.Collections.Generic;
using System.Threading.Tasks;

using Azure.Identity;
using Azure.AI.Projects;
using Azure.Core;
using Azure.Core.Pipeline;

In [8]:
internal class CustomHeadersPolicy : HttpPipelineSynchronousPolicy
{
    public override void OnSendingRequest(HttpMessage message)
    {
        message.Request.Headers.Add("x-ms-enable-preview", "true");
    }
}

In [9]:
    public sealed class SearchPlugin
    {
        [KernelFunction, Description("Search by Bing")]
        [System.Diagnostics.CodeAnalysis.SuppressMessage("Design", "CA1024:Use properties where appropriate", Justification = "Too smart")]
        public async Task<string> Search([Description("search Item")]
            string searchItem)
        {

            var connectionString = "Your Azure AI Agent Service Connection String";

            var clientOptions = new AIProjectClientOptions();

            // Adding the custom headers policy
            clientOptions.AddPolicy(new CustomHeadersPolicy(), HttpPipelinePosition.PerCall);
            var projectClient = new AIProjectClient(connectionString, new DefaultAzureCredential(), clientOptions);

            ConnectionResponse bingConnection = await projectClient.GetConnectionsClient().GetConnectionAsync("kinfey-bing-search");
            var connectionId = bingConnection.Id;

            AgentsClient agentClient = projectClient.GetAgentsClient();

            ToolConnectionList connectionList = new ToolConnectionList
            {
                ConnectionList = { new ToolConnection(connectionId) }
            };
            BingGroundingToolDefinition bingGroundingTool = new BingGroundingToolDefinition(connectionList);

            Azure.Response<Azure.AI.Projects.Agent> agentResponse = await agentClient.CreateAgentAsync(
            model: "gpt-4-1106-preview",
            name: "my-assistant",
            instructions: "You are a helpful assistant.",
            tools: new List<ToolDefinition> { bingGroundingTool });
            Azure.AI.Projects.Agent agent = agentResponse.Value;

            // Create thread for communication
            Azure.Response<AgentThread> threadResponse = await agentClient.CreateThreadAsync();
            AgentThread thread = threadResponse.Value;

            // Create message to thread
            Azure.Response<ThreadMessage> messageResponse = await agentClient.CreateMessageAsync(
                thread.Id,
                MessageRole.User,
                "How does wikipedia explain Euler's Identity?");
            ThreadMessage message = messageResponse.Value;

            // Run the agent
            Azure.Response<ThreadRun> runResponse = await agentClient.CreateRunAsync(thread, agent);

            do
            {
                await Task.Delay(TimeSpan.FromMilliseconds(500));
                runResponse = await agentClient.GetRunAsync(thread.Id, runResponse.Value.Id);
            }
            while (runResponse.Value.Status == RunStatus.Queued
                || runResponse.Value.Status == RunStatus.InProgress);

            Azure.Response<PageableList<ThreadMessage>> afterRunMessagesResponse
                = await agentClient.GetMessagesAsync(thread.Id);
            IReadOnlyList<ThreadMessage> messages = afterRunMessagesResponse.Value.Data;

            string searchResult = "";

            // Note: messages iterate from newest to oldest, with the messages[0] being the most recent
            foreach (ThreadMessage threadMessage in messages)
            {
                Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");

                if(threadMessage.Role.ToString().ToLower()=="assistant")
                {
                    foreach (MessageContent contentItem in threadMessage.ContentItems)
                    {
                        if (contentItem is MessageTextContent textItem)
                        {
                            Console.Write(textItem.Text);
                            searchResult = textItem.Text;
                        }
                        break;
                        // Console.WriteLine();
                    }
                }
            }

            return searchResult;
        }
    }

In [10]:
const string HostName = "SeachAssistant";
const string HostInstructions = "Search information ";

In [11]:
#pragma warning disable SKEXP0110



ChatCompletionAgent agent =
            new()
            {
                Instructions = HostInstructions,
                Name = HostName,
                Kernel = kernel,
                Arguments = new KernelArguments(new AzureOpenAIPromptExecutionSettings() { FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() }),
            };

In [12]:
#pragma warning disable SKEXP0110

KernelPlugin plugin = KernelPluginFactory.CreateFromType<SearchPlugin>();
agent.Kernel.Plugins.Add(plugin);

In [13]:
ChatHistory chat = [];

In [14]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

var input = "介绍一下华南师范大学";

chat.Add(new ChatMessageContent(AuthorRole.User, input));
Console.WriteLine($"# {AuthorRole.User}: '{input}'");

# user: '介绍一下华南师范大学'


In [15]:
#pragma warning disable SKEXP0110

var agentContent = agent.InvokeAsync(chat);

In [16]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

await foreach (var message in agentContent)
{
    chat.Add(message);
    Console.WriteLine($"# {message.AuthorName}: '{message.Content}'");
}

# SeachAssistant: '华南师范大学（South China Normal University，简称华师，SCNU）是中华人民共和国广东省的一所重点大学，创建于1933年。华南师范大学是一所以教育为特色，涵盖多学科的综合性大学，在师范、教育、心理学等领域具有较强的学术地位和影响力。

### 基本信息
- **地点**：广州，广东省
- **校训**：厚德载物、止于至善
- **校园环境**：校园绿化良好，建筑风格结合了现代与传统元素。

### 学术与研究
华南师范大学设有多个学院，包括但不限于教育学院、心理学院、文学院、理学院等。学校积极参与国内外学术交流与合作，建立了多项国际合作伙伴关系。

### 学生生活
华南师范大学注重学生的全面发展，除了学术课程外，还提供丰富的课外活动，包括社团、体育活动、文化艺术活动等。学生们可以在这里享受到多样化的校园文化生活。

如需更详细的信息或特定部门的介绍，欢迎询问！'
